In [1]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.6 MB/s eta 0:00:00


In [2]:
!pip install autoreject

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np

import mne
from mne import read_epochs
from mne.decoding import Vectorizer, Scaler
mne.set_log_level('error')

from autoreject import AutoReject

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import shuffle

In [2]:
# Path to gogle drive folder with hte data
path = '/content/drive/MyDrive/Colab Notebooks/Project-TFE/files/preprocessing_ica'

fists_runs = [3, 7, 11]

participants = [i for i in range(1, 81) if i not in [88, 92, 100]]

data = []
labels = []
all_epochs_per_subject = []

for participant in participants:
  participant_data = []
  labels_data = []
  all_epochs = []
  for run in fists_runs:
    epochs = read_epochs(path + f"/S{participant:03}R{run:02}filt01_30_ICA_masref.fif")
    all_epochs.append(epochs)
    participant_data.append(epochs.get_data(copy=False))
    labels_data.append(epochs.events[:, -1])
  data.append(np.concatenate(participant_data))
  labels.append(np.concatenate(labels_data))
  all_epochs_per_subject.append(all_epochs)


In [3]:
print("lenth data: ", len(data))
print("lenth labels: ", len(labels))
print("lenth all_epochs_per_subject: ", len(all_epochs_per_subject))

lenth data:  80
lenth labels:  80
lenth all_epochs_per_subject:  80


In [4]:
data_train, data_test, labels_train, labels_test, epochs_train, epochs_test = train_test_split(data, labels, all_epochs_per_subject, test_size=0.2, random_state=42)

In [5]:
X_train = np.concatenate(data_train, axis=0)
X_test = np.concatenate(data_test, axis=0)
y_train = np.concatenate(labels_train, axis=0)
y_test = np.concatenate(labels_test, axis=0)

In [6]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

In [7]:
print(X_train.shape)
print(y_train.shape)

(5125, 64, 817)
(5125,)


In [8]:
lr_pip = make_pipeline(Vectorizer(),StandardScaler(), LogisticRegression())

In [10]:
lr_pip.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x7f460436f0a0>),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [11]:
y_pred = lr_pip.predict(X_test)

In [12]:
print(sum(y_pred == y_test), " correct over ", len(y_test))
print("Accuracy: ", accuracy_score(y_test, y_pred))

681  correct over  1278
Accuracy:  0.5328638497652582
